<center>
<h1 style="font-family: 'Times New Roman', Times, serif; font-size: 60px;">DAA</h1>
<h2 style="font-family: 'Times New Roman', Times, serif; font-size: 40px;">Pre Processing</h2>
</center>

<font face="Times New Roman">

<h3 style="font-family: 'Times New Roman'">Imports</h3>

In [1]:
# Import necessary libraries
import pandas as pd              # Import pandas for data manipulation and analysis
import numpy as np               # Import numpy for numerical computations

<h3 style="font-family: 'Times New Roman'">Load All Datasets</h3>

<!-- <h3 style="font-family: 'Times New Roman'">Load All Datasets</h3> -->

In [2]:
final = pd.read_csv("Ficheiros/Ficheiros_Pos_DataAnalysis_Before_PP/Final.csv", na_filter=False)
final_teste = pd.read_csv("Ficheiros/Ficheiros_Pos_DataAnalysis_Before_PP/Final_Teste.csv")

In [3]:
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.max_rows', 500)

<h3 style="font-family: 'Times New Roman'">Pre Processing</h3>

In [4]:
# Filter rows where 'consumption_in_period' is between 6 and 8 (inclusive)
filtered_rows = final[(final['consumption_in_period'] >= 6) & (final['consumption_in_period'] <= 8)]

# Display the resulting DataFrame
print(filtered_rows)

# Drop rows where 'consumption_in_period' is between 6 and 8 (inclusive)
final.drop(final[(final['consumption_in_period'] >= 6) & (final['consumption_in_period'] <= 8)].index, inplace=True)

                    date  normal_consume  consumption_in_period  autoconsume  \
942  2021-12-08 06:00:00             0.0                  6.978          0.0   

     injection  temp  feels_like  temp_min  temp_max  humidity  ...  month  \
942          0  7.39         4.3      7.37      7.88        93  ...     12   

     year  day_of_week  hour  isFeriado  hour_str        Data  is_weekend  \
942  2021            2     6          1     06:00  2021-12-08       False   

     Estacao_do_Ano  vacaciones  
942               0           0  

[1 rows x 25 columns]


In [5]:
final['consumo_total'] = (final['autoconsume'] + final['normal_consume'] + final['consumption_in_period']).round(2)
final_teste['consumo_total'] = (final_teste['autoconsume'] + final_teste['normal_consume'] + final_teste['consumption_in_period']).round(2)

In [6]:
mask = final['consumo_total'] != 0
final['taxa_autoconsumo'] = np.where(mask, (final['autoconsume'] / final['consumo_total']).round(2), 0)

In [7]:
mask2 = final_teste['consumo_total'] != 0
final_teste['taxa_autoconsumo'] = np.where(mask2, (final_teste['autoconsume'] / final_teste['consumo_total']).round(2), 0)

In [8]:
# Extracting the hour part as an integer
final['hour'] = final['hour_str'].str.extract('(\d+)', expand=False).astype(int)

# Creating the new column based on the conditions
final['hora_partida'] = (final['hour'] >= 7) & (final['hour'] <= 18)

# Converting the boolean results to 1 and 0
final['hora_partida'] = final['hora_partida'].astype(int)

In [9]:
# Extracting the hour part as an integer
final_teste['hour'] = final_teste['hour_str'].str.extract('(\d+)', expand=False).astype(int)

# Creating the new column based on the conditions
final_teste['hora_partida'] = (final_teste['hour'] >= 7) & (final_teste['hour'] <= 18)

# Converting the boolean results to 1 and 0
final_teste['hora_partida'] = final_teste['hora_partida'].astype(int)

In [10]:
final['injection_tentativa'] = (final['autoconsume'] - (final['normal_consume'] + final['consumption_in_period'])).round(2)
final_teste['injection_tentativa'] = (final_teste['autoconsume'] - (final_teste['normal_consume'] + final_teste['consumption_in_period'])).round(2)

In [11]:
final.columns

Index(['date', 'normal_consume', 'consumption_in_period', 'autoconsume',
       'injection', 'temp', 'feels_like', 'temp_min', 'temp_max', 'humidity',
       'rain_1h', 'clouds_all', 'weather_description', 'dew_point', 'day',
       'month', 'year', 'day_of_week', 'hour', 'isFeriado', 'hour_str', 'Data',
       'is_weekend', 'Estacao_do_Ano', 'vacaciones', 'consumo_total',
       'taxa_autoconsumo', 'hora_partida', 'injection_tentativa'],
      dtype='object')

In [12]:
final["hour_str"]

0        00:00
1        01:00
2        02:00
3        03:00
4        04:00
         ...  
10267    19:00
10268    20:00
10269    21:00
10270    22:00
10271    23:00
Name: hour_str, Length: 10271, dtype: object

In [13]:
final['date'] = pd.to_datetime(final['date'])
final_teste['date'] = pd.to_datetime(final_teste['date'])

In [14]:
final.dtypes

date                     datetime64[ns]
normal_consume                  float64
consumption_in_period           float64
autoconsume                     float64
injection                         int64
temp                            float64
feels_like                      float64
temp_min                        float64
temp_max                        float64
humidity                          int64
rain_1h                         float64
clouds_all                        int64
weather_description               int64
dew_point                       float64
day                               int64
month                             int64
year                              int64
day_of_week                       int64
hour                              int32
isFeriado                         int64
hour_str                         object
Data                             object
is_weekend                         bool
Estacao_do_Ano                    int64
vacaciones                        int64


In [15]:
import ephem

# Criar colunas auxiliares
final['year'] = final['date'].dt.year
final['month'] = final['date'].dt.month
final['day'] = final['date'].dt.day
final['hour'] = final['date'].dt.hour

# Converter 'hour' para string formatada com dois dígitos
final['hour_str'] = final['hour'].apply(lambda x: f'{x:02d}')

# Converter 'hour_str' para inteiro
final['hour_str'] = final['hour_str'].astype(int)


# Função para verificar se está ensolarado
def estava_sol(row):
    observador = ephem.Observer()
    observador.lat = '41.545449'  # Substitua pela latitude desejada
    observador.lon = '-8.426507'  # Substitua pela longitude desejada

    data_hora = ephem.Date(f'{row["year"]:04d}/{row["month"]:02d}/{row["day"]:02d} {row["hour_str"]:02d}:00')
    observador.date = data_hora

    sol = ephem.Sun()
    sol.compute(observador)

    # Retorna 1 se estiver ensolarado, 0 caso contrário
    return int(sol.alt > 0)

# Aplicar a função ao DataFrame
final['is_sunny'] = final.apply(estava_sol, axis=1)

In [16]:
import ephem

# Criar colunas auxiliares
final_teste['year'] = final_teste['date'].dt.year
final_teste['month'] = final_teste['date'].dt.month
final_teste['day'] = final_teste['date'].dt.day
final_teste['hour'] = final_teste['date'].dt.hour

# Converter 'hour' para string formatada com dois dígitos
final_teste['hour_str'] = final_teste['hour'].apply(lambda x: f'{x:02d}')

# Converter 'hour_str' para inteiro
final_teste['hour_str'] = final_teste['hour_str'].astype(int)


# Função para verificar se está ensolarado
def estava_sol(row):
    observador = ephem.Observer()
    observador.lat = '41.545449'  # Substitua pela latitude desejada
    observador.lon = '-8.426507'  # Substitua pela longitude desejada

    data_hora = ephem.Date(f'{row["year"]:04d}/{row["month"]:02d}/{row["day"]:02d} {row["hour_str"]:02d}:00')
    observador.date = data_hora

    sol = ephem.Sun()
    sol.compute(observador)

    # Retorna 1 se estiver ensolarado, 0 caso contrário
    return int(sol.alt > 0)


# Aplicar a função ao DataFrame
final_teste['is_sunny'] = final_teste.apply(estava_sol, axis=1)

<h3 style="font-family: 'Times New Roman'">Save DataSet</h3>

In [17]:
final.to_csv("Ficheiros/Ficheiros_Pos_Pre_Processing/Final.csv", index = False)
final_teste.to_csv("Ficheiros/Ficheiros_Pos_Pre_Processing/Final_Teste.csv", index = False)